<a href="https://colab.research.google.com/github/tjy2202/P.T/blob/main/Side_Project/Capstone/coronary_angiography.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings(action='ignore')
from IPython.display import display
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.linear_model import LassoCV , ElasticNetCV , RidgeCV
from sklearn.preprocessing import PolynomialFeatures
from sklearn.decomposition import PCA 
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score as r2
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import GradientBoostingRegressor as GBR
from sklearn.ensemble import RandomForestRegressor as RFR
from sklearn.cross_decomposition import PLSRegression as  PLS
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline
from sklearn.model_selection import ShuffleSplit, GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.kernel_ridge import KernelRidge
import xgboost

In [ ]:
# 이름          : 
# 나이_성별     : (personal data)
# 병원 번호     :
# 검사 날짜     : 
# 비고          :
# Img_bas_n     : Basal Image number
# TFC_start_b   : (Frame count)TFC start frame_basal
# TFCC_end_b    : (Frame count)TFc end frame_basal
# CCFC_start_b  : (Frame count)CCFC start frame_basal
# CCFC_end_b    : (Frame count)CCFS end frame_basal
# Img_hyp_n     : Hyperemic Image number
# TFC_start_h   : (Frame count)TFC start frame_hyperemic
# TFC_end_h     : (Frame count)TFc end frame_hyperemic
# CCFC_start_h  : (Frame count)CCFC start frame
# CCFS_end_h    : (Frame count)CCFS end frame
# TFC_bas       : (Frame total count)TIMI Frame Count_basal
# TFC_hyp       : (Frame total count)TIMI Frame Count_hyperemic
# CCFC_bas      : (Frame total count)Coronary Clearance Frame Count_basal
# CCFC_hyp      : (Frame total count)Coronary Clearance Frame Count_hyperemic
# HR            : (personal data)Heart rate
# CFR_Therm     : Coronary Flow Reserve_Thermodilution
# CFR_TFC       : CFR by TFC
# CFR_CCFC      : CFR by CCFC
# Tmn_bas       : (Label)Transit time_basal
# Tmn_hyp       : (Label)Transit time_basal
# Pa_bas        : Aortic pressure_basal
# Pd_bas        : Distal pressure_basal 
# Pa_hyp        : Aortic pressure_hyperemic
# Pd_hyp        : Distal pressure_hyperemic
# IMR           : Index of Microcirculatory resistance
# TFC*Pa        : 
# CCFC*Pa       : 
# Pd/Pa         : Pd/Pa_basal
# FFR           : Fractional Flow Reserve_hyperemic
# PCI           : 0=no, 1=yes
# Pd/Pa_pre     : Pre-PCI Pd/Pa
# FFR_pre       : Pre-PCI FFR
# HTN           : (personal data)Hypertension
# DM            : (personal data)Diabetes
# Smoking       : (personal data)0 = ?, 1 = ?
# Dyslipidemia  : (personal data)
# CVA Hx        : (personal data)
# Stent_total   : Stent number_total
# Stent_Lesion  : Stent number_lesion
# Stent_brand   : 
# Stent_size    : Stent_size_mean
# Stent_length  : Stent_length_sum
# CAD           : 1=1VD. 2=2VD. 3=3VD. 4=LM Dz
# Lesion        : 1=LAD, 2=LCX, 3=RCA, 4=LM
# Presentation  : 1=SA, 2=UA, 3=NSTEMI,4=STEMI ,5=Atypical CP ,6=Variant angina

In [3]:
df = pd.read_excel('drive/MyDrive/project/sheet.xlsx')
df.columns = df.iloc[0,:].values
df = df.iloc[1:,1:]
df['age'], df['sex'] = np.reshape(df['나이_성별'].str.split('/').values.tolist(), (51,2)).T
require_index = ['age','sex', 'TFC_bas', 'TFC_hyp', 'CCFC_bas', 'CCFC_hyp', 'HR', 'HTN','DM','Smoking','Dyslipidemia','CVA Hx','Stent_total', 'Tmn_bas', 'Tmn_hyp']
df = df.loc[:,require_index]
df.dropna(inplace=True)
df['Stent_total'] = df['Stent_total'].replace(999,0)
df['sex'] = df['sex'].replace('M',0).replace('F',1)
df.iloc[:,[0,1,7,8,9,10,11,12]] = df.iloc[:,[0,1,7,8,9,10,11,12]].astype('category')
df.iloc[:,[2,3,4,5,6,13,14]] = df.iloc[:,[2,3,4,5,6,13,14]].astype(float)
df

,age,sex,TFC_bas,TFC_hyp,CCFC_bas,CCFC_hyp,HR,HTN,DM,Smoking,Dyslipidemia,CVA Hx,Stent_total,Tmn_bas,Tmn_hyp
1,83,0,32.0,18.0,114.0,78.0,65.0,1,0,0,0,0,2,1.30,0.48
2,54,0,38.0,18.0,92.0,29.0,80.0,0,1,1,0,0,1,0.44,0.11
3,57,0,30.0,24.0,93.0,39.0,82.0,0,1,1,0,0,2,0.49,0.20
4,59,1,22.0,12.0,85.0,38.0,73.0,0,0,0,0,0,1,0.30,0.12
5,53,0,36.0,21.0,72.0,40.0,90.0,1,1,0,1,0,0,0.52,0.19
6,73,1,60.0,28.0,103.0,38.0,72.0,0,0,0,1,0,0,0.56,0.17
7,58,0,41.0,17.0,114.0,33.0,65.0,0,0,1,1,1,1,0.61,0.16
8,69,1,22.0,11.0,51.0,20.0,62.0,0,0,0,1,0,2,0.41,0.17
9,49,0,30.0,12.0,49.0,19.0,62.0,0,0,1,1,0,1,0.54,0.21
11,76,1,27.0,16.0,62.0,28.0,76.0,1,0,0,1,0,0,0.37,0.12


In [3]:
!pip install scikit-learn==0.23.1
!pip install -q lazypredict

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.8 MB 9.8 MB/s 
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.2
    Uninstalling scikit-learn-1.0.2:
      Successfully uninstalled scikit-learn-1.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yellowbrick 1.5 requires scikit-learn>=1.0.0, but you have scikit-learn 0.23.1 which is incompatible.
imbalanced-learn 0.8.1 requires scikit-learn>=0.24, but you have scikit-learn 0.23.1 which is incompatible.


In [15]:
X_bas, X_hyp  = df.iloc[:,[0,1,2,4,6,7,8,9,10,11,12]], df.iloc[:,[0,1,3,5,6,7,8,9,10,11,12]]
y_bas, y_hyp = df.iloc[:,-2].values.reshape(-1,1), df.iloc[:,-1].values.reshape(-1,1)

X_train_bas, X_test_bas, y_train_bas, y_test_bas = train_test_split(X_bas,y_bas,test_size=.2,random_state=100)
X_train_hyp, X_test_hyp, y_train_hyp, y_test_hyp = train_test_split(X_hyp,y_hyp,test_size=.2,random_state=100)

y_train_bas = y_train_bas.reshape(-1)
y_test_bas = y_test_bas.reshape(-1)
y_train_hyp = y_train_hyp.reshape(-1)
y_test_hyp = y_test_hyp.reshape(-1)

In [10]:
from lazypredict.Supervised import LazyRegressor

clf = LazyRegressor(verbose=0, predictions=True)

models, predictions = clf.fit(X_train_bas, X_test_bas, y_train_bas, y_test_bas
models

100%|██████████| 41/41 [00:02<00:00, 18.24it/s]

[09:24:08] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


,Adjusted R-Squared,R-Squared,RMSE,Time Taken
Model,,,,
GaussianProcessRegressor,422.15,-92.59,2.10,0.02
PassiveAggressiveRegressor,77.61,-16.03,0.90,0.06
KernelRidge,29.93,-5.43,0.55,0.03
RANSACRegressor,18.49,-2.89,0.43,0.14
ExtraTreeRegressor,13.26,-1.72,0.36,0.02
DecisionTreeRegressor,11.97,-1.44,0.34,0.03
AdaBoostRegressor,11.64,-1.36,0.33,0.17
GradientBoostingRegressor,11.11,-1.25,0.33,0.07
XGBRegressor,10.45,-1.10,0.31,0.02


In [17]:
!pip install -q scikit-optimize

     |████████████████████████████████| 100 kB 7.6 MB/s 


In [ ]:


cv = ShuffleSplit(n_splits=5 , test_size=0.3, random_state=42)
pipe_xgr = Pipeline([
                ('fit', xgboost.XGBRFRegressor())])
pipe_linear = Pipeline([
                ('poly', PolynomialFeatures()),
                ('fit', LinearRegression())])
pipe_lasso = Pipeline([
                ('poly', PolynomialFeatures()),
                ('fit', Lasso(random_state = 42))])
pipe_ridge = Pipeline([
                ('poly', PolynomialFeatures()),
                ('fit', Ridge(random_state = 42))])
pipe_pca = Pipeline([
                ('pca', PCA()),
                ('fit', Ridge(random_state = 42))])
pipe_pls = Pipeline([
                ('fit', PLS())])
pipe_gbr = Pipeline([
                ('fit', GBR())])
pipe_rfr = Pipeline([
                ('fit', RFR())])
pipe_svr = Pipeline([
                ('fit', SVR())])
pipe_KR = Pipeline([
                ('fit', KernelRidge())])
                 
                 
### 
grid_params_xgr = [{
}]
grid_params_linear = [{
    "poly__degree" : np.arange(1,3), 
    "fit__fit_intercept" : [True, False], 
}]
grid_params_lasso = [{
    "poly__degree" : np.arange(1,3),
    "fit__tol" : np.logspace(-5,0,10) ,
    "fit__alpha" : np.logspace(-5,1,10) ,     
                     }]
grid_params_pca = [{
    "pca__n_components" : np.arange(2,8)
}]
grid_params_ridge = [{
    "poly__degree" : np.arange(1,3),
    "fit__alpha" : np.linspace(2,5,10) ,
    "fit__solver" : [ "cholesky","lsqr","sparse_cg"] ,
    "fit__tol" : np.logspace(-5,0,10) ,
                     }]
grid_params_pls = [{
    "fit__n_components" : np.arange(2,8)
}]
min_samples_split_range = [0.5, 0.7 , 0.9]

grid_params_gbr =[{
    "fit__max_features" : ["sqrt","log2"] ,
    "fit__loss" : ["ls","lad","huber","quantile"] , 
    "fit__max_depth" : [5,6,7,8] ,
    "fit__min_samples_split" : min_samples_split_range ,
}]
grid_params_rfr =[{
    "fit__max_features" : ["sqrt","log2"] , 
    "fit__max_depth" : [5,6,7,8] ,
    "fit__min_samples_split" : min_samples_split_range ,
}]
grid_params_svr =[{
    "fit__kernel" : ["rbf", "linear"] ,
    "fit__degree" : [2, 3, 5] , 
    "fit__gamma" : np.logspace(-5,1,10) ,
}]
grid_params_KR =[{
    "fit__kernel" : ["rbf","linear"] , 
    "fit__gamma" : np.logspace(-5,1,10) ,
}]
pipe = [
    pipe_xgr,pipe_linear , pipe_lasso ,  pipe_pca ,
    pipe_ridge , pipe_pls , pipe_gbr , 
    pipe_rfr , pipe_svr , pipe_KR 
]

params = [
    grid_params_xgr, grid_params_linear , grid_params_lasso , grid_params_pca,
    grid_params_ridge , grid_params_pls , grid_params_gbr ,
    grid_params_rfr , grid_params_svr , grid_params_KR
]

jobs = 20

grid_dict = {
    0: 'XGR',
    1: 'Linear', 
    2: 'Lasso', 
    3: 'pca regression' , 
    4: 'Ridge' ,
    5: 'PLSRegression',
    6: "GradientDescentRegressor" ,
    7: "RandomForestRegressor" ,
    8: "SupportVectorRegressor" ,
    9: "Kernel RidgeRegression"
            }

model_mse = {}
model_r2 = {}
model_best_params = {}


for idx , (param , model) in enumerate(zip(params , pipe)) :
    search = GridSearchCV(model, param, scoring  = make_scorer(mean_squared_error, greater_is_better=False), 
                          cv=cv , n_jobs=jobs , verbose=-1 )
    search.fit(X_train_bas , y_train_bas)
    y_pred = search.predict(X_test_bas)
    model_mse[grid_dict.get(idx)] = mse(y_test_bas, y_pred) ** .5
    model_r2[grid_dict.get(idx)] = r2(y_test_bas, y_pred)  
    model_best_params[grid_dict.get(idx)] = search.best_params_
print("finish")

import seaborn as sns
fig ,ax = plt.subplots(figsize=(20, 10))
sns.set(font_scale = 2)
output = pd.DataFrame([model_mse.keys() , model_mse.values()], index = ["algo","rmse"]).T
output.sort_values(["rmse"], ascending= False ,inplace=True)
ax = sns.barplot(y="algo", x="rmse", data=output)
plt.show()

In [ ]:
model = RFR()
model.fit(X_train_bas, y_train_bas)
print(f'RMSE_bsh : {mean_squared_error(y_test_bas, model.predict(X_test_bas)) ** .5:.5f}')
plt.figure(figsize=(16,8))

df = pd.DataFrame([X.columns, model.feature_importances_]).T
df.sort_values(1,ascending=False, inplace=True)

plt.barh(df[0], df[1])
plt.show()
model1 = RFR()
model1.fit(X_train_hyp, y_train_hyp)
print(f'RMSE_hyp : {mean_squared_error(y_test_hyp, model1.predict(X_test_hyp)) ** .5:.5f}')


display(pd.DataFrame([model.predict(X_test_bas), y_test_bas,model1.predict(X_test_hyp), y_test_hyp], index=['predict_bas','y_true_bas','predict_hyp','y_true_hyp']).T)